In [484]:
import requests
# url = "https://allianceforcoffeeexcellence.org/ethiopia-2022/"
url = "https://allianceforcoffeeexcellence.org/brazil-2020/"
# url = "https://cupofexcellence.org/brazil-2020/"
# url = "https://allianceforcoffeeexcellence.org/rwanda-2014/"
# url = "https://cupofexcellence.org/directory/90-03-16/"
# url = "https://farmdirectory.cupofexcellence.org/listing/legese-ethiopia-2022/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import time
import re

url = "https://allianceforcoffeeexcellence.org/brazil-2020/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
#ace&coe 全体
def scrapingPages(soup):
    result={}
    result['program']=soup.find('h1').get_text().strip()
    result['description'] = [{'p':[x.text for x in block.find_all('p')], 'li':[x.text for x in block.find_all('li')]} for block in soup.select('h1 ~ div > div.mk-text-block')]
    result['remarks']=[x.text for x in soup.select('div.vc_tta-container ~ div p')]

    def extractInfoFromPanels(panels):
        last_line_checker=re.compile('total|stat')
        schema_fixer=re.compile('[^a-zA-Z0-9]')
        for panel in panels:
            title=panel.find('span', attrs={'class': 'vc_tta-title-text'}).get_text()
            title=schema_fixer.sub('_', title)
            if 'sponsor' in title.lower(): # sponsor
                result[title]={'url':panel.img.get('src')}
            elif panel.table: #panelがtableのケース
                result[title]=[]
                table=panel.table #複数ある場合最初
                if table.find('td', attrs={'class': 'mtr-td-tag'}): #tdにheader情報がまっているとき
                    bool_jury=False
                    if 'jury' in title.lower():
                        group=None # juryの属性初期値
                        bool_jury=True
                    for tr in table.tbody.find_all('tr'):
                        if tr.find('td').get('data-mtr-content')==tr.find('td').text: #1行目の処理
                            continue
                        elif last_line_checker.match(tr.find('td').text.lower()): #最終行の処理
                            continue
                        row={}
                        #jury処理
                        if bool_jury:
                            if tr.th: # jury属性が存在する場合
                                group=tr.th.text
                                continue
                            if group:
                                row['group']=group

                        #
                        for td in tr.find_all('td'):
                            col=schema_fixer.sub('_', td.get('data-mtr-content'))
                            row[col]={}
                            row[col]['text']=td.text
                            if td.get('data-sheets-value'):
                                row[col]['value']= td.get('data-sheets-value')
                            if td.a:
                                row['url']=td.a.get('href')
                                row['result']=extractInfoFromIndividuals(row['url'])
                        result[title].append(row) #write row

                elif table.thead: #theadがあるパターン
                    cols=list(map(lambda x: schema_fixer.sub('_', x.text), table.thead.find_all('th')))
                    for tr in table.tbody.find_all('tr'):
                        if last_line_checker.match(tr.find('td').text.lower()): #最終行の処理
                            continue
                        row={}
                        for col, td in zip(cols, tr.find_all('td')):
                            row[col]= td.text
                            if td.a:
                                row['url']=td.a.get('href')
                                row['result']=extractInfoFromIndividuals(row['url'])
                        result[title].append(row) #write row

    #panels1
    panels = soup.find_all('div', attrs={'class': 'vc_tta-panel'}) #テーブルリスト
    extractInfoFromPanels(panels)
    #panels2
    a = soup.select('div.wprt-container div')[0].find(href=re.compile("https://cupofexcellence.org"))
    if a:
        url0=a.get('href')
        response0 = requests.get(url0)
        soup0 = BeautifulSoup(response0.text, 'lxml')
        panels0 = soup0.find_all('div', attrs={'class': 'vc_tta-panel'}) #テーブルリスト
        extractInfoFromPanels(panels0)
        
    return result
#coe 個別
def extractInfoFromIndividuals(url):
    individual_result={}
    time.sleep(1.5)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    if ('cupofexcellence.org/directory' in url) or ('allianceforcoffeeexcellence.org/farm-directory' in url):
        #image-links
        images = list(map(lambda x: x.img.get('srcset'), soup.select('ul.other-images a')))
        d = re.compile(r'\s\d+w')
        s = re.compile(r'\s|w')
        def chooseBestLink(x):
            linklist=x.split(',')
            resolutions=[int(s.sub('', d.search(l).group())) for l in linklist]
            maxindex=resolutions.index(max(resolutions))
            links=d.sub('', linklist[maxindex].strip())
            if len(links)==0:
                return None
            else:
                return links
        images_result=[]
        for l in [chooseBestLink(x) for x in images]:
            images_result.append(l)
        individual_result['images']=images_result
        
        #description
        description = soup.select('#listing-description')
        individual_result['description'] = {}
        individual_result['description']['p']=[p.text.strip() for p in description[0].find_all('p')]
        individual_result['description']['tr']=[tr.text.strip() for tr in description[0].find_all('tr')]
        individual_result['description']['li']=[li.text.strip() for li in description[0].find_all('li')]
        
        #detail
        schema_fixer=re.compile('[^a-zA-Z0-9]')
        details = soup.select('#listing-details tr')
        individual_table = {}
        for tr in details:
            individual_table[schema_fixer.sub('_', tr.th.text)]=tr.td.text
        individual_result['detail']=individual_table
        return(individual_result)
    elif 'farmdirectory.cupofexcellence.org/listing' in url:
        print(2)
#全体+個別
result=scrapingPages(soup)

In [2]:
import pprint
pprint.pprint(result)

{'COE_Auction_Results': [{'Company_Name': {'text': 'Luce Coffee Roasters',
                                           'value': '{"1":2,"2":"Luce Coffee '
                                                    'Roasters"}'},
                          'Farm': {'text': 'Sítio Escondica',
                                   'value': '{"1":2,"2":"Sítio Escondica"}'},
                          'High_Bid': {'text': '$40.00',
                                       'value': '{"1":3,"3":40}'},
                          'Rank': {'text': '1a', 'value': '{"1":2,"2":"1a"}'},
                          'Score': {'text': '90.03',
                                    'value': '{"1":3,"3":90.03}'},
                          'Total_Value': {'text': '$15,873.20',
                                          'value': '{"1":3,"3":15873.2}'},
                          'Weight__lbs_': {'text': '396.83',
                                           'value': '{"1":3,"3":396.83}'}},
                         {'Company_Name'

In [3]:
result

{'program': 'Brazil 2020',
 'description': [{'p': ['Cup of Excellence: The winning lots are the top 30 coffees that scored above an 87 by the Cup of Excellence International Jury. These coffees have each been cupped a minimum of five different times during the cupping process.',
    'Cup of Excellence: The winning lots are the top 30 coffees that scored above an 87 by the Cup of Excellence International Jury. These coffees have each been cupped a minimum of five different times during the cupping process. The coffee is packaged in a 30kg box containing two 15kg vacuum-packed bags.',
    'National Winners: The National Winner lots are coffees scoring 86+ points during the International Week by the Cup of Excellence International Jury. These coffees have each been cupped a minimum of five different times during the cupping process.',
    'COE Auction: December 10th\nNW Auction: December 9th – 30th'],
   'li': ['Preselection stage: September 7th – 14th with the 8 National Judges in Vargin

### test

In [498]:
#全体ページ
result={}
result['program']=soup.find('h1').get_text().strip()
result['description'] = [{'p':[x.text for x in block.find_all('p')], 'li':[x.text for x in block.find_all('li')]} for block in soup.select('h1 ~ div > div.mk-text-block')]
result['remarks']=[x.text for x in soup.select('div.vc_tta-container ~ div p')]

def extractInfoFromPanels(panels):
    last_line_checker=re.compile('total|stat')
    schema_fixer=re.compile('[^a-zA-Z0-9]')
    for panel in panels:
        title=panel.find('span', attrs={'class': 'vc_tta-title-text'}).get_text()
        title=schema_fixer.sub('_', title)
        if 'sponsor' in title.lower(): # sponsor
            result[title]={'url':panel.img.get('src')}
        elif panel.table: #panelがtableのケース
            result[title]=[]
            table=panel.table #複数ある場合最初
            if table.find('td', attrs={'class': 'mtr-td-tag'}): #tdにheader情報がまっているとき
                bool_jury=False
                if 'jury' in title.lower():
                    group=None # juryの属性初期値
                    bool_jury=True
                for tr in table.tbody.find_all('tr'):
                    if tr.find('td').get('data-mtr-content')==tr.find('td').text: #1行目の処理
                        continue
                    elif last_line_checker.match(tr.find('td').text.lower()): #最終行の処理
                        continue
                    row={}
                    #jury処理
                    if bool_jury:
                        if tr.th: # jury属性が存在する場合
                            group=tr.th.text
                            continue
                        if group:
                            row['group']=group
                    
                    #
                    for td in tr.find_all('td'):
                        col=schema_fixer.sub('_', td.get('data-mtr-content'))
                        row[col]={}
                        row[col]['text']=td.text
                        if td.get('data-sheets-value'):
                            row[col]['value']= td.get('data-sheets-value')
                        if td.a:
                            row['url']=td.a.get('href')
                    result[title].append(row) #write row
                        
            elif table.thead: #theadがあるパターン
                cols=list(map(lambda x: schema_fixer.sub('_', x.text), table.thead.find_all('th')))
                for tr in table.tbody.find_all('tr'):
                    if last_line_checker.match(tr.find('td').text.lower()): #最終行の処理
                        continue
                    row={}
                    for col, td in zip(cols, tr.find_all('td')):
                        row[col]= td.text
                        if td.a:
                            row['url']=td.a.get('href')
                    result[title].append(row) #write row

#panels1
panels = soup.find_all('div', attrs={'class': 'vc_tta-panel'}) #テーブルリスト
extractInfoFromPanels(panels)
#panels2
a = soup.select('div.wprt-container div')[0].find(href=re.compile("https://cupofexcellence.org"))
if a:
    url0=a.get('href')
    response0 = requests.get(url0)
    soup0 = BeautifulSoup(response0.text, 'lxml')
    panels0 = soup0.find_all('div', attrs={'class': 'vc_tta-panel'}) #テーブルリスト
    extractInfoFromPanels(panels0)

In [501]:
#individual-page
import time

url = "https://cupofexcellence.org/directory/90-03-16/"
# url = "https://allianceforcoffeeexcellence.org/farm-directory/89-90-3/"
# url = "https://farmdirectory.cupofexcellence.org/listing/legese-ethiopia-2022/"
def extractInfoFromIndividuals(url):
    individual_result={}
    time.sleep(1.5)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    if ('cupofexcellence.org/directory' in url) or ('allianceforcoffeeexcellence.org/farm-directory' in url):
        #image-links
        images = list(map(lambda x: x.img.get('srcset'), soup.select('ul.other-images a')))
        d = re.compile(r'\s\d+w')
        s = re.compile(r'\s|w')
        def chooseBestLink(x):
            linklist=x.split(',')
            resolutions=[int(s.sub('', d.search(l).group())) for l in linklist]
            maxindex=resolutions.index(max(resolutions))
            links=d.sub('', linklist[maxindex].strip())
            if len(links)==0:
                return None
            else:
                return links
        images_result=[]
        for l in [chooseBestLink(x) for x in images]:
            images_result.append(l)
        individual_result['images']=images_result
        
        #description
        description = soup.select('#listing-description')
        individual_result['description'] = {}
        individual_result['description']['p']=[p.text.strip() for p in description[0].find_all('p')]
        individual_result['description']['tr']=[tr.text.strip() for tr in description[0].find_all('tr')]
        individual_result['description']['li']=[li.text.strip() for li in description[0].find_all('li')]
        
        #detail
        details = soup.select('#listing-details tr')
        individual_table = {}
        for tr in details:
            individual_table[tr.th.text]=tr.td.text
        individual_result['detail']=individual_table
        return(individual_result)
    elif 'farmdirectory.cupofexcellence.org/listing' in url:
        print(2)
r=extractInfoFromIndividuals(url)

1
{'description': {'li': [],
                 'p': ['In 1984, the prdoucer’s grandfather, Sebastião '
                       'Fernandes de Sousa, together with his eight children, '
                       'purchased a farm in the city of Venda Nova do '
                       'Imigrante, ES. The farm has fertile land and a climate '
                       'favorable to coffee growing. In 2000, Luiz Ricardo’s '
                       'father, José Luiz Pimenta de Sousa, took over '
                       'production of pulped natural coffees to increase the '
                       'quality and value of the coffees. Following '
                       'Sebastião’s passing in 2008, the property was divided '
                       'among the brothers. Luiz Ricardo’s father received the '
                       'part dedicated to coffee production.',
                       'Today Luiz Ricardo and his two brothers, all youths '
                       'under 30 years of age, represent the fa

In [394]:
url = "https://cupofexcellence.org/wp-content/uploads/2020/12/whatsapp-image-2020-10-12-at-20.20.40.jpeg"
response = requests.get(url)
image = response.content
with open('./images/'+url, "wb") as f:
    f.write(image)

FileNotFoundError: [Errno 2] No such file or directory: './images/https://cupofexcellence.org/wp-content/uploads/2020/12/whatsapp-image-2020-10-12-at-20.20.40.jpeg'

In [466]:
import json
f = open('test.json', 'w') #書き込むファイルを開く
json.dump(result, f) #変数 2 は辞書型

In [500]:
f = open('test.json', 'w')
# f.write(json.dumps(result['Winning Farms*']))
f.write(json.dumps(result))
# f.write(json.dumps({key: result[key] for key in ['Winning Farms*']}))
f.close()